### Проверяем выбран ли у нас GPU

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

### Удаляем ненужную папку sample_data

In [ ]:
!rm -rf ./sample_data

### Устанавливаем утилиты

Клонируем репозиторий и меняем в файле general_json2yolo.py меняем 274 строчку на <br>h, w, f = img['height'], img['width'], img['file_name'].split('/')[1]</br>

В файлах general_json2yolo.py и labelbox_json2yolo.py, добавьте в перед каждым utils точку, чтобы было так <br>from .utils import ...

In [ ]:
!git clone https://github.com/ultralytics/JSON2YOLO

### Устанавливаем библиотеки

In [ ]:
%pip install -r /content/JSON2YOLO/requirements.txt

In [ ]:
%pip install kaggle ipywidgets widgetsnbextension

### Скачиваем датасет

In [ ]:
import os

In [ ]:
os.environ['KAGGLE_USERNAME'] = "xxx"
os.environ['KAGGLE_KEY'] = "xxx"
!kaggle datasets download -d watchman/rtsd-dataset

### Распаковываем архив

In [ ]:
import zipfile

In [ ]:
archive = zipfile.ZipFile('rtsd-dataset.zip', 'r')
archive.extractall('.')

### Удаляем архив

In [ ]:
os.remove('rtsd-dataset.zip')

### Импортируем библиотеки

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from shutil import copyfile, move
import sys
import json
from ipywidgets import FloatProgress

In [ ]:
from JSON2YOLO.general_json2yolo import convert_coco_json
sys.path.append('./JSON2YOLO')

test_path = 'test_annotation'
train_path = 'train_annotation'

os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

In [ ]:
move('train_anno.json', os.path.join(train_path, 'train_anno.json'))
move('val_anno.json', os.path.join(test_path, 'val_anno.json'))

In [ ]:
for folder in ['labels', 'images']:
    for path in [test_path, train_path]:
        os.makedirs(os.path.join(path, folder), exist_ok=True)

In [ ]:
convert_coco_json(train_path)
for file in tqdm(os.listdir(os.path.join('new_dir/labels/train_anno'))):
    move(os.path.join('new_dir/labels/train_anno', file), os.path.join(train_path, 'labels', file))

convert_coco_json('./test_annotation/')
for file in tqdm(os.listdir(os.path.join('new_dir/labels/val_anno'))):
    move(os.path.join('new_dir/labels/val_anno', file), os.path.join(test_path, 'labels', file))

### Добавим изображения уже к готовой разметке

In [ ]:
test_labels = os.listdir(os.path.join(test_path, 'labels'))
train_labels = os.listdir(os.path.join(train_path, 'labels'))

test_labels = set(map(lambda x: x.split('.')[0], test_labels))
train_labels = set(map(lambda x: x.split('.')[0], train_labels))

In [ ]:
images = 'rtsd-frames/rtsd-frames'
for file in os.listdir(images):
    name = file.split('.')[0]
    if name in train_labels:
        move(os.path.join(images, file), os.path.join(train_path,'images', file))
    if name in test_labels:
        move(os.path.join(images, file), os.path.join(test_path,'images', file))

### Создадим yaml файл

In [ ]:
%pip install PyYAML==5.1

In [ ]:
import yaml

In [ ]:
data = [{
    'train': '/content/train_annotation/images',
    'val': '/content/test_annotation/images',
    'nc': 155,
    'names': ['2_1', '1_23', '1_17', '3_24', '8_2_1', '5_20', '5_19_1', '5_16', '3_25',
              '6_16', '7_15', '2_2', '2_4', '8_13_1', '4_2_1', '1_20_3', '1_25', '3_4', '8_3_2',
              '3_4_1', '4_1_6', '4_2_3', '4_1_1', '1_33', '5_15_5', '3_27', '1_15', '4_1_2_1',
              '6_3_1', '8_1_1', '6_7', '5_15_3', '7_3', '1_19', '6_4', '8_1_4', '8_8', '1_16',
              '1_11_1', '6_6', '5_15_1', '7_2', '5_15_2', '7_12', '3_18', '5_6', '5_5', '7_4',
              '4_1_2', '8_2_2', '7_11', '1_22', '1_27', '2_3_2', '5_15_2_2', '1_8', '3_13',
              '2_3', '8_3_3', '2_3_3', '7_7', '1_11', '8_13', '1_12_2', '1_20', '1_12', '3_32',
              '2_5', '3_1', '4_8_2', '3_20', '3_2', '2_3_6', '5_22', '5_18', '2_3_5', '7_5',
              '8_4_1', '3_14', '1_2', '1_20_2', '4_1_4', '7_6', '8_1_3', '8_3_1', '4_3', '4_1_5',
              '8_2_3', '8_2_4', '1_31', '3_10', '4_2_2', '7_1', '3_28', '4_1_3', '5_4', '5_3',
              '6_8_2', '3_31', '6_2', '1_21', '3_21', '1_13', '1_14', '2_3_4', '4_8_3', '6_15_2',
              '2_6', '3_18_2', '4_1_2_2', '1_7', '3_19', '1_18', '2_7', '8_5_4', '5_15_7', '5_14',
              '5_21', '1_1', '6_15_1', '8_6_4', '8_15', '4_5', '3_11', '8_18', '8_4_4', '3_30',
              '5_7_1', '5_7_2', '1_5', '3_29', '6_15_3', '5_12', '3_16', '1_30', '5_11', '1_6',
              '8_6_2', '6_8_3', '3_12', '3_33', '8_4_3', '5_8', '8_14', '8_17', '3_6', '1_26',
              '8_5_2', '6_8_1', '5_17', '1_10', '8_16', '7_18', '7_14', '8_23']
}]

In [ ]:
def write_yaml_to_file(py_obj, filename) :
    with open(f'{filename}.yaml', 'w+',) as f:
        yaml.dump_all(py_obj, f, sort_keys=False)
write_yaml_to_file(data, 'trafic_signs')

### Устанавливаем библиотеки

In [ ]:
%pip install ultralytics

### Обучение

In [ ]:
from ultralytics import YOLO
import torch
import gc

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model = YOLO('yolov8s.pt')

results = model.train(
   data='/content/trafic_signs.yaml',
   imgsz=1280,
   epochs=5,
   batch=6,
   device=0,
   name='YOLOv8s'
)